In [1]:
import os
import torch
from datasets import load_dataset, Dataset
import pandas as pd
import transformers
from transformers import AutoTokenizer
from trl import SFTTrainer
import transformers
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from pynvml import *
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import glob

In [2]:
from huggingface_hub import login

In [3]:
# login()

In [4]:
directory_path = r'/home/sdm/Desktop/shakib/KUET LLM/data/'
file_pattern = '*.docx'  
file_paths = glob.glob(directory_path + file_pattern)
file_paths

['/home/sdm/Desktop/shakib/KUET LLM/data/rag4.docx',
 '/home/sdm/Desktop/shakib/KUET LLM/data/rag2V2.docx',
 '/home/sdm/Desktop/shakib/KUET LLM/data/rag1.docx']

In [5]:
def splitter_ret():
    length_function = len
    splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=500, 
    chunk_overlap=100,
    length_function=length_function,
)
    return splitter

In [6]:

def create_vectorstore(flag):
    if flag == False:
        flag=True
        # loader = Docx2txtLoader(r"data/Khulna University of Engineering.docx")
        # documents=loader.load()
        all_doc=[]
        for x in file_paths:
            loader = Docx2txtLoader(x)
            documents=loader.load()
            all_doc.extend(documents)
        # text_splitter=splitter_ret()
        text_splitter=CharacterTextSplitter(chunk_size=500,chunk_overlap=30,separator="\n")
        docs=text_splitter.split_documents(documents=all_doc)
        # text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        # docs = text_splitter.split_documents(documents)
        model_name = "sentence-transformers/all-mpnet-base-v2"
        hf = HuggingFaceEmbeddings(model_name=model_name)
        vectorstore=FAISS.from_documents(docs,hf)
        vectorstore.save_local('vectorstore')
        return vectorstore
    else:
        model_name = "sentence-transformers/all-mpnet-base-v2"
        hf = HuggingFaceEmbeddings(model_name=model_name)
        new_vectorstore=FAISS.load_local("vectorstore",hf)
        return new_vectorstore

In [7]:
vectorstore=create_vectorstore(False)

Created a chunk of size 854, which is longer than the specified 500
Created a chunk of size 576, which is longer than the specified 500
Created a chunk of size 723, which is longer than the specified 500
Created a chunk of size 751, which is longer than the specified 500
Created a chunk of size 570, which is longer than the specified 500
Created a chunk of size 737, which is longer than the specified 500
Created a chunk of size 634, which is longer than the specified 500
Created a chunk of size 632, which is longer than the specified 500
Created a chunk of size 585, which is longer than the specified 500
Created a chunk of size 542, which is longer than the specified 500
Created a chunk of size 922, which is longer than the specified 500
Created a chunk of size 595, which is longer than the specified 500
Created a chunk of size 587, which is longer than the specified 500
Created a chunk of size 626, which is longer than the specified 500
Created a chunk of size 523, which is longer tha

In [8]:
from langchain import HuggingFacePipeline
# path='google/gemma-7b'
path="KUETLLM_zepyr"
tokenizer = AutoTokenizer.from_pretrained(path,
                                          use_auth_token=True,)

# model = AutoModelForCausalLM.from_pretrained(path,
#                                              device_map='auto',
#                                              torch_dtype=torch.float16,
#                                              use_auth_token=True,
#                                              load_in_8bit=True,
#                                             #  load_in_4bit=True
#                                              )

model = AutoModelForCausalLM.from_pretrained(
    path,
    # quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,

)
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )
    
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0})


/home/sdm/anaconda3/envs/llmtesting/lib/python3.9/site-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
retriever=vectorstore.as_retriever()

In [10]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [11]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
# torch.cuda.empty_cache()

In [12]:
def ans_ret(inp,history):
    ans=rag_chain.invoke(inp)
    k=ans.split("Based on the text material")
    k2=ans.split("Hope that helped! Let me know if you have any more questions.")

    if len(k)>=2:
        k3=k[0].split("Hope that helped! Let me know if you have any more questions.")
        if len(k3)>=2:
            return k3[0]
        else:
            return k[0]
    if len(k2)>=2:
        return k2[0]
    return ans

In [13]:
import gradio as gr
questions=[
    'How many halls in KUET?', 'weather of KUET',"What is the most beautiful things in KUET", "IS KUET RAGGING FREE?","Why KUET CSE is called CSE Family? "
]
demo = gr.ChatInterface(fn=ans_ret, examples=questions, title="KUET information Bot")
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Running on public URL: https://be9d598fb7ad66b31e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/sdm/anaconda3/envs/llmtesting/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/sdm/anaconda3/envs/llmtesting/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/sdm/anaconda3/envs/llmtesting/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during 